# COVID-19: Main protease binding site defined by XChem ligands

Aim of this notebook:
    
- Load all XChem structures in Biopython
- Get residues in radius cutoff of ligand centroids
- Find overlapping residues across all structures (by residue coverage threshold)

## Data

> To contribute to the global effort to combat COVID-19, Diamond has been able to solve a new structure of the SARS-CoV-2 main protease (MPro) at high resolution (PDB ID: 6YB7), and complete a large XChem crystallographic fragment screen against it (detailed below). Data have been deposited with the PDB, but we are making the results available immediately to the world on this page; additional work is ongoing, and updates will be continually posted here in coming days and weeks.

https://www.diamond.ac.uk/covid-19/for-scientists/Main-protease-structure-and-XChem.html 

## Binding site definition

Transform the following cell to Code cell if you need to install packages.

In [1]:
from collections import Counter
from pathlib import Path

from Bio.PDB import PDBParser, Selection, NeighborSearch
import pandas as pd

Globals used in this notebook:

In [2]:
# Ligand name in dataset (the same name in all structures, thanks!)
LIGAND_NAME = 'H_LIG'

# Radius from ligand centroid to end of binding site
DISTANCE_CUTOFF = 10

# Percentage of structures that need to show a certain residue ID in their binding site
COVERAGE_CUTOFF = 0.5  

In [3]:
# Path to folder with structures
STRUCTURE_FOLDER = Path('.') / '..' / 'data' / 'Mpro_All_PDBs'

# Get path to all structure files
STRUCTURE_PATHS = [pdb for pdb in STRUCTURE_FOLDER.glob('*.pdb')]

In [4]:
N_STRUCTURES = len(STRUCTURE_PATHS) 
print(f'Number of structures: {N_STRUCTURES}')

Number of structures: 78


Functions used in this notebook:

In [5]:
def get_ligand(chain, ligand_name):
    """
    Get ligand from Diamond structures.
    """
    
    for residue in chain.get_residues():
        
        if residue.full_id[3][0] == ligand_name:
            return residue

In [6]:
def get_centroid(residue):
    """
    Get centroid for residue atoms.
    """

    coordinates = pd.DataFrame(
        [atom.get_coord() for atom in residue.get_atoms()],
        columns='x y z'.split()
    )

    return list(coordinates.mean())

In [7]:
def binding_site_residues(structure_path, ligand_name, distance_cutoff):
    """
    Get binding site residues (Biopython objects) from Diamond structure.
    """
    
    # Read structure
    parser = PDBParser()
    structures = parser.get_structure(structure_path.stem, structure_path)

    # Extract protein and ligand
    protein = structures[0]['A']  # Includes DMS and LIG but egal
    ligand = get_ligand(structures[0]['A'], ligand_name)

    # Get residues around ligand centroid
    atoms  = Selection.unfold_entities(protein, 'A')
    ns = NeighborSearch(atoms)
    closest_residues = ns.search(get_centroid(ligand), distance_cutoff, 'R')
    
    return closest_residues

In [8]:
def mulitple_binding_site_residue_ids(structure_paths, ligand_name, distance_cutoff):
    """
    Get binding site IDs from a set of Diamond structures.
    """

    residue_ids = {}

    for structure_path in structure_paths:

        closest_residues = binding_site_residues(structure_path, ligand_name, distance_cutoff)
        closest_residue_ids = [residue.full_id[3][1] for residue in closest_residues if residue.full_id[3][0] == ' ']
        
        residue_ids[structure_path.stem] = closest_residue_ids
        
    return residue_ids

In [9]:
def binding_site_residue_coverage(residue_ids, n_structures, coverage_cutoff):
    """
    Get binding site residues from a set of PDB structures, which are greater or equal to a given coverage cutoff.
    """
    
    residue_ids_flat = []
    for residues in residue_ids.values():
        residue_ids_flat = residue_ids_flat + residues

    counter = Counter(residue_ids_flat)
    coverage = pd.DataFrame(counter.items(), columns=['residue_id', 'n_structures'])
    coverage['coverage'] = coverage.n_structures / n_structures
    
    return coverage[coverage.coverage >= coverage_cutoff]

Get binding site residues passing a defined coverage cutoff.

In [10]:
residue_ids = mulitple_binding_site_residue_ids(STRUCTURE_PATHS, LIGAND_NAME, DISTANCE_CUTOFF)
coverage = binding_site_residue_coverage(residue_ids, N_STRUCTURES, COVERAGE_CUTOFF)
coverage

,residue_id,n_structures,coverage
0,142,58,0.743590
2,42,47,0.602564
4,49,54,0.692308
6,141,51,0.653846
7,26,44,0.564103
9,143,52,0.666667
10,164,58,0.743590
11,145,58,0.743590
13,41,57,0.730769
15,44,51,0.653846
